<h1>CERTIFICATION PROJET : SCRIPT ENEDIS DATA</h1>

In [1]:
# importation des librairies
import csv
import pandas as pd

<h1>I- importation du csv</h1>

In [2]:
# Je crée une variable file dans laquelle j'ouvre le csv téléchargé depuis mon compte Enedis
file = pd.read_csv('C:/Users/x/Documents/Projet simplon/Certification Data/Code/Enedis/ERDF_ODO_HDM_20190716221656_1024179125@07586251793436@CDC@19-9a20de41@160720192116.csv', sep=';')

file.head()

,Identifiant PRM,Type de donnees,Date de debut,Date de fin,Grandeur physique,Grandeur metier,Etape metier,Unite,Pas en minutes
0,07586251793436,Courbe de charge,16/07/2017,16/07/2019,Energie active,Consommation,Comptage Brut,W,NaN
1,Horodate,Valeur,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-02-09T00:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-02-09T01:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-02-09T02:00:00+01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h1>II- Nettoyage pour un dataframe opérationel</h1>

**Méthodologie**


Pour mon analyse, j'ai besoin uniquement des données de temps 'Identifiant PRM' et de consommation 'Type de donnees'.
<br>Mon compteur Linky a été installé au cours du mois de Février 2019 à mon domicile. Je ne me souviens pas précisément de la date, je vais donc avoir des données manquantes et il va falloir sélectionner les lignes et les colonnes dont j'ai besoin, puis faire un traitement pour rendre le dataframe opérationnel.
<br>Il y a donc des valeurs manquantes à éliminer.

<h2>A- Sélection des données</h2>

<h3>1) Sélection des colonnes utiles pour l'analyse sans les valeurs manquantes</h3>

In [3]:
# Je vérifie le nom des colonnes
file.columns

Index(['Identifiant PRM', 'Type de donnees', 'Date de debut', 'Date de fin',
       'Grandeur physique', 'Grandeur metier', 'Etape metier', 'Unite',
       'Pas en minutes'],
      dtype='object')

Les premières lignes représentent les entêtes et des lignes avec des valeurs manquantes, j'en ai pas besoin.
<br>Je vais donc faire une boucle afin de connaître à partir de quelle ligne/quelle date je dispose de données.

In [4]:
# Je vérifie le shape de file
file.shape

(7512, 9)

In [5]:
# J'ai d'abord besoin de connaître le nombre total de ligne dans le dataframe file:
nb_ligne = file.shape[0]
print (nb_ligne)

7512


In [6]:
# BOUCLE pour connaître la première ligne avec une consommation
# tant que a est inférieur à 1, on reste dans la boucle
# puis on regarde si le type de donnée i est bien une chaîne de caractère str sinon on passe à la ligne suivante
a=0
i = 2 # parceque les 2 premières lignes sont des entêtes
while a < 1:
    if type(file['Type de donnees'][i]) is str:
        print(i)
        a=1
    elif i==file.shape[0]:
        a=1
    else: 
        i=i+1

# on affiche le dataframe créé à partir de la première ligne sans donnée manquante NaN, jusqu'à la fin.       
df = file[['Identifiant PRM', 'Type de donnees']].loc[i:]
df.head()

26


,Identifiant PRM,Type de donnees
26,2019-02-10T00:00:00+01:00,68
27,2019-02-10T00:00:00+01:00,68
28,2019-02-10T01:00:00+01:00,127
29,2019-02-10T01:00:00+01:00,127
30,2019-02-10T02:00:00+01:00,50


Je remarque donc que j'ai des données effectifs à partir du 10 février (ligne 26)

In [7]:
# Récupération des données à partir du 10 février et pour les colonnes 'Identifiant PRM' et 'Type de donnees'
df_1002 = file[['Identifiant PRM', 'Type de donnees']].loc[26:]

In [8]:
# un extrait
df_1002.head()

,Identifiant PRM,Type de donnees
26,2019-02-10T00:00:00+01:00,68
27,2019-02-10T00:00:00+01:00,68
28,2019-02-10T01:00:00+01:00,127
29,2019-02-10T01:00:00+01:00,127
30,2019-02-10T02:00:00+01:00,50


In [9]:
# un autre extrait
df_1002.tail()

,Identifiant PRM,Type de donnees
7507,2019-07-15T21:30:00+02:00,2922
7508,2019-07-15T22:00:00+02:00,1514
7509,2019-07-15T22:30:00+02:00,854
7510,2019-07-15T23:00:00+02:00,760
7511,2019-07-15T23:30:00+02:00,698


Donc je possède des données du 10 février 2019 (2019-02-10T00:00:00+01:00)au 15 juillet 2019 (2019-07-15T23:30:00+02:00)

In [10]:
# Je vérifie le shape
df_1002.shape

(7486, 2)

In [11]:
# Je vérifie le nom des colonnes
df_1002.columns

Index(['Identifiant PRM', 'Type de donnees'], dtype='object')

In [12]:
# Je vérifie le type des colonnes
df_1002.dtypes

Identifiant PRM    object
Type de donnees    object
dtype: object

<h3>2) Sélection des données pour des step de 1 heure</h3>

In [13]:
#Sur l'ensemble des lignes, je récupère uniquement un pas de 2, d'où [::2]
df_1002 = df_1002[::2]

In [14]:
# Je vérifie un extrait du datafrme
df_1002.head()

,Identifiant PRM,Type de donnees
26,2019-02-10T00:00:00+01:00,68
28,2019-02-10T01:00:00+01:00,127
30,2019-02-10T02:00:00+01:00,50
32,2019-02-10T03:00:00+01:00,214
34,2019-02-10T04:00:00+01:00,119


In [15]:
# Je vérifie la taille du dataframe
df_1002.shape

(3743, 2)

<h2>B- Traitement des données</h2>

Le dataframe df_1002 a ses colonnes de type object. Il faut que je les transforme en type datetime et integer

<h3>1) Transformation type 'Identifiant PRM'</h3>

**Méthodologie**
<br>J'utilise une REGEX(= expression régulière) afin de transformer le format de 'Identifiant PRM' sur un format qui sera reconnu dans SQL.

In [16]:
# pour rappel, le format de 'date_hour' avant la REGEX
df_1002['Identifiant PRM'].head()

26    2019-02-10T00:00:00+01:00
28    2019-02-10T01:00:00+01:00
30    2019-02-10T02:00:00+01:00
32    2019-02-10T03:00:00+01:00
34    2019-02-10T04:00:00+01:00
Name: Identifiant PRM, dtype: object

In [17]:
# changement des données de la colonne 'Identifiant PRM' suite à la REGEX
df_1002['Identifiant PRM'] = df_1002['Identifiant PRM'].str.replace(r'...:..$','').replace(r'^(....\-..\-..)T', r'\1 ', regex=True)

In [18]:
# Je vérifie le changement
df_1002['Identifiant PRM'].head()

26    2019-02-10 00:00:00
28    2019-02-10 01:00:00
30    2019-02-10 02:00:00
32    2019-02-10 03:00:00
34    2019-02-10 04:00:00
Name: Identifiant PRM, dtype: object

In [19]:
# changement du type 'Identifiant PRM'
df_1002['Identifiant PRM'] = pd.to_datetime(df_1002['Identifiant PRM'])

<h3>2) Vérification du  type 'Type de donnees' (</h3>

In [20]:
# un rappel du type des colonnes
df_1002.dtypes

Identifiant PRM    datetime64[ns]
Type de donnees            object
dtype: object

Les valeurs manquantes NaN sont considérées de type float.
<br> il faut d'abord que je traite les NaN puis je remplace le type de la colonne en INTEGER.

In [21]:
# Je cherche la somme des NaN dans l'ensemble des colonnes du dataframe
df_1002.isna().sum()

Identifiant PRM     0
Type de donnees    24
dtype: int64

Il y a 24 données manquantes dans la colonne 'Type de donnees'
<br> Comme je ne peux pas transformer le type de la colonnes 'type de donnees' en INTEGER car il y a des NaNS, il faut d'abord que je supprime les NaNs.

<h3>3) Suppression des valeurs manquantes</h3>

In [22]:
# Je crée un nouveau dataframe nan_rows qui comporte les données null présent dans le 
nan_rows = df_1002[df_1002['Type de donnees'].isnull()]

In [23]:
# Je vérifie la taille du dataframe
nan_rows.shape 

(24, 2)

J'ai 24 lignes, qui représente bien les 24 valeurs manquantes

In [24]:
# J'affiche ce dataframe
nan_rows

,Identifiant PRM,Type de donnees
5546,2019-06-05 01:00:00,NaN
5548,2019-06-05 02:00:00,NaN
5550,2019-06-05 03:00:00,NaN
5552,2019-06-05 04:00:00,NaN
5554,2019-06-05 05:00:00,NaN
5556,2019-06-05 06:00:00,NaN
5558,2019-06-05 07:00:00,NaN
5560,2019-06-05 08:00:00,NaN
5562,2019-06-05 09:00:00,NaN
5564,2019-06-05 10:00:00,NaN


Les 24 lignes représentent toutes les 24 heures de la journée du 5 Juin 2019. J'en déduis qu'il y a eu un bugg et que mon compteur linky n'a pas pu communiquer ma consommation à Enedis.

In [25]:
# Je supprime donc les 24 lignes dans l'ancien le dataframe df_1002
df_enedisConso = pd.concat([df_1002, nan_rows]).drop_duplicates(keep=False)

In [26]:
# Je vérifie ce nouveau dataframe
df_enedisConso.head()

,Identifiant PRM,Type de donnees
26,2019-02-10 00:00:00,68
28,2019-02-10 01:00:00,127
30,2019-02-10 02:00:00,50
32,2019-02-10 03:00:00,214
34,2019-02-10 04:00:00,119


In [27]:
# Je vérifie qu'il n'ya plus de valeurs manquantes
df_enedisConso.isna().sum()

Identifiant PRM    0
Type de donnees    0
dtype: int64

<h3>4) Transformation du type de 'Types de donnees'</h3>

In [28]:
# Je transforme le type de la colonnes 'Type de donnees' en float
df_enedisConso['Type de donnees'] = df_enedisConso['Type de donnees'].astype(int)

In [29]:
# Je vérifie de nouveau le type des colonnnes du dataframe
df_enedisConso.dtypes

Identifiant PRM    datetime64[ns]
Type de donnees             int32
dtype: object

<h3>3) Renommage des colonnes</h3>

In [30]:
df_enedisConso = df_enedisConso.rename(index=str, columns={"Identifiant PRM":"date_hour", "Type de donnees":"consumption_hour"})

In [31]:
# Je vérifie un extrait du dataframe
df_enedisConso.head(30)

,date_hour,consumption_hour
26,2019-02-10 00:00:00,68
28,2019-02-10 01:00:00,127
30,2019-02-10 02:00:00,50
32,2019-02-10 03:00:00,214
34,2019-02-10 04:00:00,119
36,2019-02-10 05:00:00,34
38,2019-02-10 06:00:00,237
40,2019-02-10 07:00:00,91
42,2019-02-10 08:00:00,52
44,2019-02-10 09:00:00,127


<h1>III- Exportation en csv</h1>

In [32]:
df_enedisConso.to_csv('C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/CertificationData/fournisseur/ConsoEnedis_df_1002.csv', index=False)